In [7]:
import cv2, glob, numpy as np
import os

pattern_sizes = [(9,6), (7,5), (10,7)]

square_size = 1.0  

datasets = {
    "webcam": r"calib_webcam\*.jpg",
    "dslr":   r"calib_phonecam\*.jpg"
}


In [8]:
def load_images(path_pattern):
    images = glob.glob(path_pattern)
    print(f"📂 Found {len(images)} images in {path_pattern}")
    return images


In [12]:
def calibrate_camera(images, pattern_size, dataset_name):
    objpoints, imgpoints = [], []
    
    objp = np.zeros((pattern_size[0]*pattern_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2) * square_size
    
    gray = None
    good_images = 0
    
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)

        print(fname, "Corners found?", ret)

        if ret:
            good_images += 1
            corners = cv2.cornerSubPix(
                gray, corners, (11,11), (-1,-1),
                (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,30,0.001)
            )
            objpoints.append(objp)
            imgpoints.append(corners)

    if len(objpoints) == 0:
        print(f"❌ No corners detected with pattern_size={pattern_size} in {dataset_name}")
        return None


    ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, gray.shape[::-1], None, None
    )


    total_err = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], K, dist)
        err = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
        total_err += err
    mean_err = total_err / len(objpoints)

    print(f"\n✅ Calibration successful for {dataset_name} with pattern_size={pattern_size}")
    print("Camera Matrix (K):\n", K)
    print("Distortion Coefficients:\n", dist.ravel())
    print("Mean Reprojection Error (px):", mean_err)
    return {
        "K": K,
        "dist": dist,
        "error": mean_err
    }


In [13]:
results = {}

for pattern_size in pattern_sizes:
    for dataset_name, path in datasets.items():
        images = load_images(path)
        if len(images) > 0:
            res = calibrate_camera(images, pattern_size, dataset_name)
            results[(dataset_name, pattern_size)] = res



📂 Found 17 images in calib_webcam\*.jpg
calib_webcam\img_00.jpg Corners found? True
calib_webcam\img_01.jpg Corners found? True
calib_webcam\img_02.jpg Corners found? True
calib_webcam\img_03.jpg Corners found? True
calib_webcam\img_04.jpg Corners found? True
calib_webcam\img_05.jpg Corners found? True
calib_webcam\img_06.jpg Corners found? True
calib_webcam\img_07.jpg Corners found? True
calib_webcam\img_08.jpg Corners found? True
calib_webcam\img_09.jpg Corners found? True
calib_webcam\img_10.jpg Corners found? True
calib_webcam\img_11.jpg Corners found? True
calib_webcam\img_12.jpg Corners found? True
calib_webcam\img_13.jpg Corners found? True
calib_webcam\img_14.jpg Corners found? True
calib_webcam\img_15.jpg Corners found? True
calib_webcam\img_16.jpg Corners found? True

✅ Calibration successful for webcam with pattern_size=(9, 6)
Camera Matrix (K):
 [[617.60630727   0.         328.96711945]
 [  0.         618.99545631 240.23130232]
 [  0.           0.           1.        ]]
Dis

In [14]:
print("\n📊 Calibration Results Summary:")
for key, res in results.items():
    if res:
        dataset_name, pattern_size = key
        print(f"{dataset_name} | Pattern {pattern_size} | Error: {res['error']:.4f}px")



📊 Calibration Results Summary:
webcam | Pattern (9, 6) | Error: 0.2235px
dslr | Pattern (9, 6) | Error: 0.2539px
dslr | Pattern (7, 5) | Error: 0.1335px
